In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

### Loading

In [2]:
DATA_ROOT = Path("../../data")
TARGET_DATA = "IMDB Dataset.csv"

data = pd.read_csv(DATA_ROOT / "raw/imdb" / TARGET_DATA)
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
data["sentiment"] = data["sentiment"].replace({
  "negative": 0,
  "positive": 1,
})

/tmp/u/kopyla/ipykernel_27697/2859058455.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["sentiment"] = data["sentiment"].replace({


### Processing

In [4]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [5]:
import nltk
from nltk.tokenize import ToktokTokenizer

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

In [6]:
import re
from bs4 import BeautifulSoup


def denoise_text(text: str) -> str:
    soup = BeautifulSoup(text, "html.parser")

    tmp = soup.get_text()
    return re.sub(r"\[[^]]*\]", '', tmp)


data["review"] = data["review"].apply(denoise_text)

In [ ]:
def remove_special_characters(text: str) -> str:
    pattern=r'[^a-zA-z\s]'
    return re.sub(pattern, '', text)


data["review"] = data["review"].apply(remove_special_characters)

In [8]:
def simple_stemmer(text: str) -> str:
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

data["review"] = data["review"].apply(simple_stemmer)

In [9]:
data

,review,sentiment
0,one of the other review ha mention that after ...,1
1,a wonder littl product the film techniqu is ve...,1
2,i thought thi wa a wonder way to spend time on...,1
3,basic there a famili where a littl boy jake th...,0
4,petter mattei love in the time of money is a v...,1
...,...,...
49995,i thought thi movi did a down right good job i...,1
49996,bad plot bad dialogu bad act idiot direct the ...,0
49997,i am a cathol taught in parochi elementari sch...,0
49998,im go to have to disagre with the previou comm...,0


In [10]:
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
print(stop)

def remove_stopwords(text: str) -> str:
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]

    return ' '.join(filtered_tokens)    


data["review"] = data["review"].apply(remove_stopwords)

{'too', 'doesn', 'has', 'or', 'while', 'again', 'they', 'from', 'no', "he'll", 'before', 'which', 'in', 'an', 'during', 'that', 'you', "haven't", "she'd", 'some', 'about', 'had', 'herself', 'now', "you've", 'having', 'needn', "you'll", 'at', 'themselves', 'by', "it's", 'did', "i'll", 'these', 'and', 'because', 'but', 'such', 'my', 'when', 'with', 'hadn', "couldn't", 'him', 'yourself', "we'll", "we'd", 'ourselves', "it'd", 'down', 'theirs', 'he', "i'm", 'just', 'over', 'weren', 'myself', 'does', "hasn't", 'ma', 'above', 'whom', "she'll", 'for', 'was', 'being', 'other', "don't", "you'd", 'hers', 'how', 'haven', 'between', 'off', 'what', "didn't", 'its', "mightn't", 'any', 'she', "we've", "wasn't", 'all', "i've", 'under', 'their', 'i', 'have', "it'll", 'yourselves', "they'd", 'there', 'didn', 'most', "we're", 'm', "mustn't", 'couldn', 'same', "she's", 'can', 'so', 'of', 'mustn', 'very', "wouldn't", "shan't", 'doing', "they'll", 'will', 's', 'our', 're', 'below', 'were', 'll', 'into', 'wou

In [ ]:
#normalized train reviews
# norm_train_reviews=imdb_data.review[:40000]
# norm_train_reviews[0]

#convert dataframe to string
# norm_train_string=norm_train_reviews.to_string()

#Spelling correction using Textblob
# norm_train_spelling=TextBlob(norm_train_string)
#norm_train_spelling.correct()

#Tokenization using Textblob
#norm_train_words=norm_train_spelling.words
#norm_train_words

data

,review,sentiment
0,one review ha mention watch oz episod youll ho...,1
1,wonder littl product film techniqu veri unassu...,1
2,thought thi wa wonder way spend time hot summe...,1
3,basic famili littl boy jake think zombi hi clo...,0
4,petter mattei love time money visual stun film...,1
...,...,...
49995,thought thi movi right good job wasnt creativ ...,1
49996,bad plot bad dialogu bad act idiot direct anno...,0
49997,cathol taught parochi elementari school nun ta...,0
49998,im go disagre previou comment side maltin thi ...,0


### Vectorization

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(
    # min_df=0.0, 
    # max_df=1.0,
    # ngram_range=(1,3),
)
embedded = vect.fit_transform(data["review"])

In [32]:
vect.get_feature_names_out()[600:700]

array(['abrupt', 'abruptli', 'abruptlydirect', 'abruptnessther',
       'absalom', 'abscbn', 'abscess', 'abscond', 'abseil', 'absenc',
       'absencein', 'absenceth', 'absens', 'absent', 'absentalic',
       'absentbut', 'absente', 'absenther', 'absentia', 'absentiaand',
       'absentmi', 'absentmind', 'absentminded', 'absentmindedli',
       'absentmindedyetcap', 'absentoh', 'absentsubplot', 'absentth',
       'absinth', 'abskani', 'absolom', 'absolout', 'absolu', 'absolut',
       'absolutelli', 'absolutelyconfus', 'absolutelyfantast',
       'absolutelysham', 'absolutelyugh', 'absolutey', 'absolutl',
       'absolutley', 'absolutli', 'absolv', 'absorb', 'absorbedlik',
       'absorbingli', 'absorpt', 'absoul', 'absoulutley', 'absout',
       'absoutley', 'abstain', 'abstin', 'abstinencethi', 'abstract',
       'abstractsurrealparallel', 'abstrus', 'absurd', 'absurda',
       'absurdabout', 'absurdand', 'absurdanesthesiologist',
       'absurdanoth', 'absurdbut', 'absurddiari', 'ab

In [33]:
embedded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4915462 stored elements and shape (50000, 174913)>

### Data dump

In [34]:
data = embedded.toarray()
target = data["sentiment"]

with open(DATA_ROOT / "processed/climate" / TARGET_DATA.replace(".csv", ".npz"), "xb") as f:
    np.savez(
        f,
        data=data,
        target=target,
        spatial=data,
    ) 

MemoryError: Unable to allocate 65.2 GiB for an array with shape (50000, 174913) and data type float64

In [35]:
embedded.shape

(50000, 174913)